<a href="https://colab.research.google.com/github/CrisMcode111/DI_Bootcamp/blob/main/w7_d2_Pinecone_Serverless_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daily Challenge: Pinecone Serverless Reranking in Action


Why are we doing this?
Reranking models boost search relevance by assigning similarity scores between a query and documents, then
reordering results so the most pertinent information appears first. In contexts like healthcare, this helps clinicians
quickly access the most critical clinical notes.



Task Overview & Detailed Explanations
Below is a skeleton pipeline. Each numbered item is an action you must complete. After every instruction, you’ll find a
clear explanation of what to do and why it’s important. Whenever you see … , replace it with the appropriate code or
value, using the hint for guidance.

⚠️ Important: Make sure you have a Pinecone account and API key ready. Sign up at pinecone if you haven’t already.

In [27]:
# Part 1: Load Documents & Execute Reranking Model
!pip install -U pinecone==6.0.1 pinecone-notebooks

In [29]:
import os
from pinecone_notebooks.colab import Authenticate

if not os.environ.get("PINECONE_API_KEY"):
    print("Please enter your Pinecone API key when prompted below to proceed.")
    Authenticate()


In [30]:
import os
from pinecone import Pinecone

# IMPORTANT: Replace 'YOUR_API_KEY_HERE' with your actual Pinecone API key.
# You can find your API key in your Pinecone dashboard.
api_key_manual = "pcsk_3HQSWw_KzH26gXMvkS3JPWcCFHvobaDgBzMU1TZbjV2yjs6p5Rr4j7NmwE4t7HdnLJy45f" # <--- REPLACE THIS WITH YOUR ACTUAL API KEY!

# Set the environment variable directly, or pass it to the constructor
os.environ["PINECONE_API_KEY"] = api_key_manual

pc = Pinecone(api_key=api_key_manual)

print("Pinecone client initialized successfully!")

Pinecone client initialized successfully!


In [31]:
import os
from pinecone import Pinecone

api_key = os.environ.get("PINECONE_API_KEY") # Corrected to use the standard environment variable name
pc = Pinecone(api_key=api_key)


In [32]:
query = "Tell me about Apple's products"

documents = [
    #  Add a document about apple fruit
    "The apple is a sweet fruit that comes in many varieties, including Fuji and Granny Smith. People eat apples fresh, baked in pies, or blended into juices.",

    # Add a document about Apple company products
    "Apple Inc. is a technology company known for its innovative products such as the iPhone, iPad, MacBook, and Apple Watch, used worldwide.",

    # Add another fruit-related document
    "Apples grow on trees and are rich in fiber and vitamins. They are often used to make cider, applesauce, and healthy snacks for children.",

    # Add another company-related document
    "Apple also develops software products like iOS, macOS, and services such as iCloud, Apple Music, and the App Store, forming a complete digital ecosystem.",

    ## Add one more document my choice
    "Although apple the fruit is popular in desserts, Apple Inc. creates electronics like AirPods and Mac computers. The two meanings of 'apple' often confuse learners of English."
]


In [33]:
from pinecone import RerankModel
reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
    top_n=4
)

In [36]:
print(reranked)
if reranked and reranked.results:
    print(f"Number of reranked results: {len(reranked.results)}")
else:
    print("Reranking returned no results or an empty object.")

RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=1,
    score=0.9528382,
    document={
        id='1',
        text='Apple Inc. is a technology company known for its innovative products such as the iPhone, iPad, MacBook, and Apple Watch, used worldwide.'
    }
  },{
    index=3,
    score=0.8677098,
    document={
        id='3',
        text='Apple also develops software products like iOS, macOS, and services such as iCloud, Apple Music, and the App Store, forming a complete digital ecosystem.'
    }
  },{
    index=4,
    score=0.21568096,
    document={
        id='4',
        text="Although apple the fruit is popular in desserts, Apple Inc. creates electronics like AirPods and Mac computers. The two meanings of 'apple' often confuse learners of English."
    }
  },{
    index=2,
    score=0.04169577,
    document={
        id='2',
        text='Apples grow on trees and are rich in fiber and vitamins. They are often used to make cider, applesauce, and healthy snacks 

In [37]:
def show_reranked_results(query, matches):
    print(f"Query: {query}")
    for i, m in enumerate(matches):
        print(f"{i+1}. Score: {m.score:.4f}, Text: {m.document.text}")

show_reranked_results(query, reranked.data)

Query: Tell me about Apple's products
1. Score: 0.9528, Text: Apple Inc. is a technology company known for its innovative products such as the iPhone, iPad, MacBook, and Apple Watch, used worldwide.
2. Score: 0.8677, Text: Apple also develops software products like iOS, macOS, and services such as iCloud, Apple Music, and the App Store, forming a complete digital ecosystem.
3. Score: 0.2157, Text: Although apple the fruit is popular in desserts, Apple Inc. creates electronics like AirPods and Mac computers. The two meanings of 'apple' often confuse learners of English.
4. Score: 0.0417, Text: Apples grow on trees and are rich in fiber and vitamins. They are often used to make cider, applesauce, and healthy snacks for children.


In [38]:
# Part 2: Setup a Serverless Index for Medical Notes
!pip install pandas torch transformers

In [39]:
import os
import time
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
import torch

# Get cloud and region settings (these are defaults that work for most users)
cloud = os.getenv('PINECONE_CLOUD', 'aws') # e.g., 'aws'
region = os.getenv('PINECONE_REGION', 'us-east-1') # e.g., 'us-east-1'

# Define serverless specifications
spec = ServerlessSpec(cloud=cloud, region=region)

# Define index name
index_name = 'my-reranking-index' # Give your index a name
print(f"Pinecone cloud: {cloud}, region: {region}, index name: {index_name}")

Pinecone cloud: aws, region: us-east-1, index name: my-reranking-index


In [40]:
# Clean up any existing index with the same name
if pc.has_index(name=index_name):
    pc.delete_index(name=index_name)

# Create a new index
pc.create_index(
    name=index_name,
    dimension=384, # This matches our embedding model size
    metric='cosine', # Distance metric for similarity
    spec=spec
)

{
    "name": "my-reranking-index",
    "metric": "cosine",
    "host": "my-reranking-index-heviqew.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [43]:
# Part 3: Load the Sample Data
import requests
import tempfile

with tempfile.TemporaryDirectory() as tmpdirname:
    file_path = os.path.join(tmpdirname, "sample_notes_data.jsonl")

    # Download the file from github
    url = "https://raw.githubusercontent.com/pinecone-io/examples/refs/heads/master/docs/data/sample_notes_data.jsonl" # Insert the GitHub raw URL here
    response = requests.get(url)
    response.raise_for_status()

    with open(file_path, "wb") as f:
        f.write(response.content)

    df = pd.read_json(file_path, orient='records', lines=True)

In [44]:
# Show head of the DataFrame
print("Data shape:", df.shape) # Show number of rows and columns
df.head()

Data shape: (100, 3)


,id,values,metadata
0,P011,"[-0.2027486265, 0.2769146562, -0.1509393603, 0...","{'advice': 'rest, hydrate', 'symptoms': 'heada..."
1,P001,"[0.1842793673, 0.4459365904, -0.0770567134, 0....","{'tests': 'EKG, stress test', 'symptoms': 'che..."
2,P002,"[-0.2040648609, -0.1739618927, -0.2897160649, ...","{'HbA1c': '7.2', 'condition': 'diabetes', 'med..."
3,P003,"[0.1889383644, 0.2924542725, -0.2335938066, -0...","{'symptoms': 'cough, wheezing', 'diagnosis': '..."
4,P004,"[-0.12171068040000001, 0.1674752235, -0.231888...","{'referral': 'dermatology', 'condition': 'susp..."


In [46]:
# Part 4: Upsert Data into the Index

# Instantiate an index client
index = pc.Index(name=index_name)

# Upsert data into index from DataFrame
index.upsert_from_dataframe(df) # Pass the DataFrame

sending upsert requests:   0%|          | 0/100 [00:00<?, ?it/s]

{'upserted_count': 100}

In [47]:
def is_fresh(index):
    stats = index.describe_index_stats()
    vector_count = stats.total_vector_count
    print(f"Vector count: ", vector_count)
    return vector_count > 0 # What should this be?

while not is_fresh(index):
    time.sleep(5)

print("Index ready!")
index.describe_index_stats()

Vector count:  100
Index ready!


{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}

In [56]:
# Part 5: Query & Embedding Function
import torch
from transformers import AutoTokenizer, AutoModel

def get_embedding(input_question):
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    encoded_input = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        # Average pool last hidden states to get a single vector for the sentence
        # Correctly average across the sequence length dimension (dim=1 of the original output, or dim=0 after [0])
        embedding = model_output.last_hidden_state[0].mean(dim=0)
    return embedding

In [57]:
# Build a query to search
question = "Did you have a heart attack before?" # Ask a medical question
query = get_embedding(question).tolist()

# Get results
results = index.query(vector=[query], top_k=4, include_metadata=True)

# Sort results by score in descending order
sorted_matches = sorted(results['matches'], key=lambda x: x['score'], reverse=True)

In [60]:
# Part 6: Display & Rerank Clinical Notes
def show_results(question, matches):
    print(f'Question: \'{question}\'')
    print('\nResults:')
    for i, match in enumerate(matches):
        print(f'{str(i+1).rjust(4)}. ID: {match["id"]}')
        print(f' Score: {match["score"]}') # What field contains the score?
        print(f' Metadata: {match["metadata"]}') # What field contains metadata?
        print('')

show_results(question, sorted_matches)

Question: 'Did you have a heart attack before?'

Results:
   1. ID: P001
 Score: 0.389990896
 Metadata: {'symptoms': 'chest pain', 'tests': 'EKG, stress test'}

   2. ID: P016
 Score: 0.369224429
 Metadata: {'condition': 'heart murmur', 'referral': 'cardiology'}

   3. ID: P089
 Score: 0.3023974
 Metadata: {'advice': 'monitor blood pressure', 'vital_signs': 'slightly elevated'}

   4. ID: P041
 Score: 0.3023974
 Metadata: {'advice': 'monitor blood pressure', 'vital_signs': 'slightly elevated'}



In [70]:
# Create documents with concatenated metadata field as "reranking_field" field
transformed_documents = [
    {
        'id': match['id'],
        'reranking_field': '; '.join([f"{key}: {value}" for key, value in match['metadata'].items()]),
        'text': '; '.join([f"{key}: {value}" for key, value in match['metadata'].items()]) # Add text field for display
    }
    for match in results['matches']
]

In [71]:
# Define a more specific query for reranking
refined_query = "patient needs knee surgery" # Make a more specific medical question

# Perform reranking based on the query and specified field
reranked_results = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=refined_query,
    documents=transformed_documents,
    rank_fields=["reranking_field"],
    top_n=3, # How many top results do you want?
    return_documents=True
)

In [72]:
def display_reranked_clinical_notes(query, reranked_data):
    print(f"Reranked for query: '{query}'")
    print("\nTop reranked clinical notes:")
    for i, res in enumerate(reranked_data):
        print(f"{i+1}. ID: {res.document.id}")
        print(f"   Score: {res.score:.4f}")
        print(f"   Content: {res.document.text}")
        print("")

display_reranked_clinical_notes(refined_query, reranked_results.data)

Reranked for query: 'patient needs knee surgery'

Top reranked clinical notes:
1. ID: P001
   Score: 0.0010
   Content: symptoms: chest pain; tests: EKG, stress test

2. ID: P089
   Score: 0.0003
   Content: advice: monitor blood pressure; vital_signs: slightly elevated

3. ID: P041
   Score: 0.0003
   Content: advice: monitor blood pressure; vital_signs: slightly elevated



In [73]:
def show_reranked_results(question, matches):
    print(f'Question: \'{question}\'')
    print('\nReranked Results:')
    for i, match in enumerate(matches):
        print(f'{str(i+1).rjust(4)}. ID: {match.document.id}')
        print(f' Score: {match.score:.4f}') # What attribute contains the reranking score?
        print(f' Reranking Field: {match.document.text}') # What contains the searchable field?
        print('')
show_reranked_results(refined_query, reranked_results.data) # What attribute contains the results?

Question: 'patient needs knee surgery'

Reranked Results:
   1. ID: P001
 Score: 0.0010
 Reranking Field: symptoms: chest pain; tests: EKG, stress test

   2. ID: P089
 Score: 0.0003
 Reranking Field: advice: monitor blood pressure; vital_signs: slightly elevated

   3. ID: P041
 Score: 0.0003
 Reranking Field: advice: monitor blood pressure; vital_signs: slightly elevated



In [76]:
# Delete the index to save resources
from pinecone import NotFoundException
try:
    pc.delete_index(name=index_name)
    print(f"Index '{index_name}' deleted successfully.")
except NotFoundException:
    print(f"Index '{index_name}' not found, so no deletion was performed.")
except Exception as e:
    print(f"An unexpected error occurred while deleting index '{index_name}': {e}")

Index 'my-reranking-index' not found, so no deletion was performed.
